In [0]:
# Create a dimension specifically for 15-minute intervals
time_quarters = []
for hour in range(24):
    for quarter in range(4):
        minutes = quarter * 15
        time_str = f"{hour:02d}:{minutes:02d}:00"
        
        row = {
            "time_quarter_id": hour * 4 + quarter,  # Primary key (0-95)
            "hour": hour,                           # Hour (0-23)
            "quarter_of_hour": quarter,             # Quarter (0-3)
            "minute": minutes,                      # Minute (0, 15, 30, 45)
            "time_15min": time_str,                 # Time string (HH:MM:SS)
            "time_display": f"{hour:02d}:{minutes:02d}",  # Display format
            "hour_12": ((hour - 1) % 12) + 1,       # 12-hour format (1-12)
            "am_pm": "AM" if hour < 12 else "PM",   # AM/PM indicator
            "time_period": "Morning" if 6 <= hour < 12 else 
                           "Afternoon" if 12 <= hour < 18 else 
                           "Evening" if 18 <= hour < 22 else "Night",
            "is_business_hour": 1 if 9 <= hour < 18 and quarter > 0 else 0,  # 9:15 AM to 5:45 PM
            "sort_order": hour * 4 + quarter        # For ordering (0-95)
        }
        time_quarters.append(row)

df_time_quarter = spark.createDataFrame(time_quarters)


In [0]:
external_path = "abfss://gold@fingridtest.dfs.core.windows.net/gold/dim_time/"
df_time_quarter.write.format("delta").option("mergeSchema", "true").mode("overwrite").save(external_path)




In [0]:
%sql
CREATE TABLE IF NOT EXISTS fingrid_test_workspace.fingrid_gold.dim_time
USING DELTA
LOCATION 'abfss://gold@fingridtest.dfs.core.windows.net/gold/dim_time/'

In [0]:
%sql
select * from fingrid_test_workspace.fingrid_gold.dim_time